In [1]:
gpu_id = 0

In [2]:
import os, sys
import numpy as np
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

sys.path.append("../../road")
from data import load_det_dataset
from models import get_model_detection
from evaluation import evaluate
from helpers import train_epoch_linear, lr_schedule, Monitor, evaluate_coco, get_coco_stat
from config import model_path as model_dir

%load_ext autoreload
%autoreload 2

### Experiment parameters

In [3]:
batch_size = 8
num_workers = 8
device = f"cuda:{gpu_id}"

In [4]:
model_name = "resnet152"

In [5]:
nepochup = 1
momentum = 0.9
weight_decay=0.0005

In [6]:
mode = "class"
maxlr = .005
num_epochs = 20

### Initialization

In [7]:
# Data
data_loader, data_loader_test = load_det_dataset(mode=mode, 
                                                 batch_size=batch_size, 
                                                 num_workers=num_workers)
num_classes = data_loader.dataset.nclass()
total_steps = num_epochs*len(data_loader)

In [8]:
print(num_classes)

5


In [9]:
# Dumps
monitor   = Monitor(f"{mode}/OneCycle_{maxlr}_nepoch_{num_epochs}")
model_dir = f"{model_dir}/{mode}/OneCycle_{maxlr}_nepoch_{num_epochs}"
if not os.path.isdir(model_dir):
    os.makedirs(model_dir);

In [10]:
# Model
model  = get_model_detection(num_classes, model_name=model_name).to(device)
for p in model.parameters():
    p.requires_grad_(True)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=1,
                            momentum=momentum, 
                            weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=maxlr, total_steps=total_steps)
print(f"Finetuning (faster rcnn) {len(params)} parameters out of {len(list(model.parameters()))}")

Finetuning (faster rcnn) 185 parameters out of 185


In [ ]:
for epoch in range(num_epochs):
    print(f"Start training epoch {epoch} ...")
    train_epoch_linear(model, optimizer, data_loader, monitor, lr_scheduler=lr_scheduler)
    print("Start evaluation ...")
    tem = evaluate_coco(model, data_loader_test)
    monitor.update(get_coco_stat(tem))
    dt_eval, eval_data = evaluate(model, data_loader_test)
    monitor.update(eval_data)
    torch.save(model.state_dict(), f"{model_dir}/epoch_{epoch}")

  0%|          | 0/2266 [00:00<?, ?it/s]

Start training epoch 0 ...


/home/tristan/anaconda3/envs/road/lib/python3.8/site-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629395347/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)
  6%|▋         | 147/2266 [02:42<39:06,  1.11s/it]